In [1]:
from PIL import Image, ImageOps
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

In [2]:
def prepareImage(file):
    original_image = Image.open(file)
    title = original_image.crop((0, 0, 600, 55))
    x_ticks = original_image.crop((74,301,600,337))
    x_title = original_image.crop((53,338,600,368))
    y_title = original_image.crop((0,0,400,400)).rotate(270).crop((0,0,350,38))
    bottom_y_tick = original_image.crop((38,280,80,310))
    y_ticks = original_image.crop((38,77,84,307))
    top_y_tick = original_image.crop((38,77,94,96))
    legend = original_image.crop((2,48,599,74))
    num = (pytesseract.image_to_string(top_y_tick))
    newnum = ""
    for i in num:
        if i.isdigit():
            newnum = newnum + i
    if len(newnum)>3:
        top_y_tick_bound = (pytesseract.image_to_boxes(top_y_tick)).split()
        y_ticks = original_image.crop((38,77,84+(84-int(top_y_tick_bound[-3])),310))
        bottom_y_tick = original_image.crop((38,280,80+(80-int(top_y_tick_bound[-3])),310))
    return original_image,title,x_ticks,y_ticks,x_title,y_title,bottom_y_tick,top_y_tick,legend

In [38]:
file = r"C:\Users\local1\sarvghad visualizations\Evaluation Graphs\COVID-19\2Bar\COVID vs flu cases, 2020 10.png"
original_image,title,x_ticks,y_ticks,x_title,y_title,bottom_y_tick,top_y_tick,legend = prepareImage(file)

In [4]:
text_from_image = list(pytesseract.image_to_string(x_ticks))
output = []
previous = 0
for i in range(len(text_from_image)):
    global previous
    if (text_from_image[i]=='\n'):
        output.append(text_from_image[previous:i])
        previous = i+1
if len(output)==1:
    new_output = []
    previous = 0
    for i in range(len(output[0])):
        if output[0][i] == ' ':
            new_output.append(output[0][previous:i])
            previous = i+1
            new_output.append([])
    new_output.append(output[0][previous:len(output[0])])
    output = new_output
print(output)
#y_ticks.show()

[['J', 'a', 'n'], [], ['F', 'e', 'b'], [], ['M', 'a', 'r'], [], ['A', 'p', 'r'], [], ['M', 'a', 'y'], [], ['J', 'u', 'n'], [], ['J', 'u', 'l', '.'], [], ['A', 'u', 'g'], [], ['S', 'e', 'p'], [], ['O', 'c', 't'], [], ['N', 'o', 'v'], [], ['D', 'e', 'c', '.']]


In [5]:
boxes_from_image = pytesseract.image_to_boxes(x_ticks)
boxes = []
previous = 0
for i in range(len(boxes_from_image)):
    global previous
    if (boxes_from_image[i]=='\n'):
        boxes.append((boxes_from_image[previous:i]).split())
        previous = i+1
print(boxes)
#pprint.pprint(boxes)

[['J', '21', '20', '26', '29', '0'], ['a', '28', '20', '33', '27', '0'], ['n', '35', '20', '40', '27', '0'], ['F', '62', '20', '67', '29', '0'], ['e', '69', '20', '74', '27', '0'], ['b', '76', '20', '81', '29', '0'], ['M', '104', '20', '111', '29', '0'], ['a', '113', '20', '118', '27', '0'], ['r', '120', '20', '123', '27', '0'], ['A', '144', '20', '151', '29', '0'], ['p', '152', '18', '157', '27', '0'], ['r', '159', '20', '162', '27', '0'], ['M', '185', '20', '192', '29', '0'], ['a', '194', '20', '199', '27', '0'], ['y', '200', '18', '205', '27', '0'], ['J', '225', '20', '230', '29', '0'], ['u', '232', '20', '237', '27', '0'], ['n', '239', '20', '244', '27', '0'], ['J', '268', '20', '273', '29', '0'], ['u', '275', '20', '280', '27', '0'], ['l', '282', '20', '283', '29', '0'], ['.', '282', '20', '283', '29', '0'], ['A', '306', '20', '313', '29', '0'], ['u', '314', '20', '319', '27', '0'], ['g', '321', '18', '326', '27', '0'], ['S', '348', '20', '354', '29', '0'], ['e', '356', '20', '361

In [6]:
x_tick_coords = {}
count = 0
for List in output:
    if len(List)>0:
        x_tick_coords[''.join(List)] = int(boxes[count][1])+5
        for i in range(len(List)):
            count+=1
print(x_tick_coords)

{'Jan': 26, 'Feb': 67, 'Mar': 109, 'Apr': 149, 'May': 190, 'Jun': 230, 'Jul.': 273, 'Aug': 311, 'Sep': 353, 'Oct': 395, 'Nov': 435, 'Dec.': 475}


In [7]:
for key in x_tick_coords:
    x_tick_coords[key] = x_tick_coords[key]+74

In [8]:
visType = "2bar"

In [9]:
#add top of bar coords to new list
if visType=="1bar":
    tops = []
    for tick in x_tick_coords:
        y = 306
        start = False
        while y != 83:
            if original_image.load()[x_tick_coords[tick],y] != (255,255,255,255):
                start = True
            if start:
                if original_image.load()[x_tick_coords[tick],y] == (255,255,255,255):
                    tops.append(y+1)
                    break
            y = y - 1
    for x in range(len(tops)):
        tops[x] = 300 - tops[x]
    print(tops)
elif visType =="1line":
    tops = []
    count = 0
    for tick in x_tick_coords:
        y = 306
        while y != 83:
            if count == 0:
                if original_image.load()[x_tick_coords[tick]+10,y] != (255, 255, 255,255):
                    tops.append(y+1)
                    break
                y = y - 1
            elif count == len(list(dict.keys(x_tick_coords)))-1:
                if original_image.load()[x_tick_coords[tick]-5,y] != (255, 255, 255,255):
                    tops.append(y+1)
                    break
                y = y - 1
            else:
                if original_image.load()[x_tick_coords[tick],y] != (255, 255, 255,255):
                    tops.append(y+1)
                    break
                y = y - 1
        count+=1
    for x in range(len(tops)):
        tops[x] = 300 - tops[x]
    print(tops)
elif visType == "2bar":
    count = 0
    for key in x_tick_coords:
        if (len(key)%2==0):
            middle_coord = (int(boxes[count][3]) + int(boxes[len(key)][3]))//2
        elif (len(key)%2!=0):
            middle_coord = (boxes[len(key)//2][3])
        count = count + len(key)
    left_tops = []
    right_tops = []
    for tick in x_tick_coords: #for left
        y = 306
        start = False
        while y != 83:
            if original_image.load()[x_tick_coords[tick]-6,y] != (255,255,255,255):
                start = True
            if start:
                if original_image.load()[x_tick_coords[tick]-6,y] == (255,255,255,255):
                    left_tops.append(y+1)
                    break
            y = y - 1
    for x in range(len(left_tops)):
        left_tops[x] = 300 - left_tops[x]
    print(left_tops)
    for tick in x_tick_coords: #for right
        y = 306
        start = False
        while y != 83:
            if original_image.load()[x_tick_coords[tick]+6,y] != (255,255,255,255):
                start = True
            if start:
                if original_image.load()[x_tick_coords[tick]+6,y] == (255,255,255,255):
                    right_tops.append(y+1)
                    break
            y = y - 1
    for x in range(len(right_tops)):
        right_tops[x] = 300 - right_tops[x]
    print(right_tops)
elif visType == "2line":
    left_tops = []
    right_tops = []
    tickCount = 0
    for tick in x_tick_coords:
        y = 306
        foundRed = False
        foundBlue = False
        foundPoints = []
        while y != 83:
            if tickCount == 0:
                if original_image.load()[x_tick_coords[tick]+10,y] == (234, 67, 53, 255) and foundRed == False:#red
                    right_tops.append(y+1)
                    foundRed = True
                    foundPoints.append(y)
                elif original_image.load()[x_tick_coords[tick]+10,y] == (66, 133, 244, 255) and foundBlue == False: #blue
                    left_tops.append(y+1)
                    foundBlue = True
                    foundPoints.append(y)
            elif tickCount == len(list(dict.keys(x_tick_coords)))-1:
                if original_image.load()[x_tick_coords[tick]-5,y] == (234, 67, 53, 255) and foundRed == False:#red
                    right_tops.append(y+1)
                    foundRed = True
                    foundPoints.append(y)
                elif original_image.load()[x_tick_coords[tick]-5,y] == (66, 133, 244, 255) and foundBlue == False: #blue
                    left_tops.append(y+1)
                    foundBlue = True
                    foundPoints.append(y)
            else:
                if original_image.load()[x_tick_coords[tick],y] == (234, 67, 53, 255) and foundRed == False:#red
                    right_tops.append(y+1)
                    foundRed = True
                    foundPoints.append(y)
                elif original_image.load()[x_tick_coords[tick],y] == (66, 133, 244, 255) and foundBlue == False: #blue
                    left_tops.append(y+1)
                    foundBlue = True
                    foundPoints.append(y)
                    
            y=y-1
        tickCount+=1
        if len(foundPoints)==1:
            if len(left_tops) < len(right_tops):
                left_tops.append(foundPoints[0]+1)
            elif len(right_tops) < len(left_tops):
                right_tops.append(foundPoints[0]+1)
    for x in range(len(left_tops)):
        left_tops[x] = 300 - left_tops[x]
    print(left_tops)
    for x in range(len(right_tops)):
        right_tops[x] = 300 - right_tops[x]
    print(right_tops)

[1, 1, 1, 19, 29, 22, 32, 65, 43, 43, 83, 169]
[17, 26, 17, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
text_from_image = list(pytesseract.image_to_string(y_ticks))
output = []
previous = 0
for i in range(len(text_from_image)):
    global previous
    if (text_from_image[i]=='\n'):
        output.append(text_from_image[previous:i])
        previous = i+1
print(output)
#y_ticks.show()
#print(pytesseract.image_to_string(y_ticks))

[['2', '0', '0'], [], ['1', '5', '0'], [], ['1', '0', '0'], [], ['5', '0']]


In [11]:
boxes_from_image = pytesseract.image_to_boxes(y_ticks)
boxes = []
previous = 0
for i in range(len(boxes_from_image)):
    global previous
    if (boxes_from_image[i]=='\n'):
        boxes.append((boxes_from_image[previous:i]).split())
        previous = i+1
print(boxes)

[['2', '14', '215', '19', '224', '0'], ['0', '21', '215', '26', '224', '0'], ['0', '28', '215', '33', '224', '0'], ['1', '14', '162', '17', '171', '0'], ['5', '21', '162', '26', '171', '0'], ['0', '28', '162', '33', '171', '0'], ['1', '14', '109', '17', '118', '0'], ['0', '21', '109', '26', '118', '0'], ['0', '28', '109', '33', '118', '0'], ['5', '21', '56', '26', '65', '0'], ['0', '28', '56', '33', '65', '0']]


In [12]:
y_tick_coords = {}
count = 0
for List in output:
    if len(List)>0:
        y_tick_coords[''.join(List)] = 322-int(boxes[count][2])+83
        for i in range(len(List)):
            count+=1
newdict = {}
for key in y_tick_coords:
    newnum = ""
    for i in key:
        if i.isdigit():
            newnum = newnum + i
    newdict[int(newnum)] = y_tick_coords[key]
y_tick_coords = newdict
print(y_tick_coords)

{200: 190, 150: 243, 100: 296, 50: 349}


In [13]:
bottom_y = int(pytesseract.image_to_string(bottom_y_tick,config="--psm 10"))
if list(dict.keys(y_tick_coords))[-1] != bottom_y:
    coord = y_tick_coords[list(dict.keys(y_tick_coords))[-1]]
    del y_tick_coords[list(dict.keys(y_tick_coords))[-1]]
    y_tick_coords[bottom_y] = coord
print(y_tick_coords)

{200: 190, 150: 243, 100: 296, 0: 349}


In [14]:
data = []
left_data = []
right_data = []
keys = list(dict.keys(y_tick_coords))
top_tick = y_tick_coords[keys[0]]
bottom_tick = y_tick_coords[keys[1]]
pixel_difference = (bottom_tick-top_tick)
tick_difference = (int(keys[0])- int(keys[1]))
unit_value = tick_difference/pixel_difference
if visType!="2bar" and visType!="2line":
    for i in range(len(tops)):
        data.append(round(unit_value*tops[i]+float(bottom_y)))
elif visType=="2bar" or visType=="2line":
    for i in range(len(left_tops)):
        left_data.append(round(unit_value*left_tops[i]+float(bottom_y)))
    for i in range(len(right_tops)):
        right_data.append(round(unit_value*right_tops[i]+float(bottom_y)))
    
print(tick_difference)
print(pixel_difference)
print(bottom_y)
print(data)
print(left_data)
print(right_data)
#bottom_y_tick.show()

50
53
0
[]
[1, 1, 1, 18, 27, 21, 30, 61, 41, 41, 78, 159]
[16, 25, 16, 1, 1, 1, 1, 1, 1, 1, 1]


In [15]:
if (visType!="2bar" and visType!="2line"):
    x_data = x_tick_coords
    keys = list(dict.keys(x_data))
    for i in range(len(data)):
        x_data[keys[i]] = data[i]
elif (visType=="2bar" or visType =="2line"):
    x_data = x_tick_coords
    keys = list(dict.keys(x_data))
    for i in range(len(left_data)):
        x_data[keys[i]] = left_data[i]
    for i in range(len(right_data)):
        x_data[keys[i]] = x_data[keys[i]],right_data[i]

In [95]:
feature_extraction = {}
feature_extraction['title'] = pytesseract.image_to_string(title)
feature_extraction['x_ticks'] = list(dict.keys(x_tick_coords))
feature_extraction['y_ticks'] = list(dict.keys(y_tick_coords))
feature_extraction['x_title'] =  pytesseract.image_to_string(x_title)
feature_extraction['y_title'] =  pytesseract.image_to_string(y_title)
feature_extraction['data'] = x_data
feature_extraction['bottom_y_tick'] = bottom_y

In [96]:
if visType=="2bar" or visType =="2line":
    #labels = pytesseract.image_to_string(legend).split("\n\n")
    #for i in range(len(labels)):
    #    labels[i] = labels[i].replace("=","")
    #    labels[i] = labels[i].replace("\n"," ")
    #print(labels)
    for i in range(599):
        if original_image.load()[i,62] == (234, 67, 53, 255):
            legend1 = original_image.crop((2,48,i,74))
            legend2 = original_image.crop((i,48,599,74))
            label1 = pytesseract.image_to_string(legend1)
            label2 = pytesseract.image_to_string(legend2)
            feature_extraction['legend'] = (label1,label2)
            print(feature_extraction['legend'])
            break

('@ COVID cases, thousands\n', 'M™ Flu cases, thousands\n')


In [101]:
import pprint
pprint.pprint(feature_extraction)

{'bottom_y_tick': 0,
 'data': {'Apr': (18, 1),
          'Aug': (61, 1),
          'Dec.': 159,
          'Feb': (1, 25),
          'Jan': (1, 16),
          'Jul.': (30, 1),
          'Jun': (21, 1),
          'Mar': (1, 16),
          'May': (27, 1),
          'Nov': (78, 1),
          'Oct': (41, 1),
          'Sep': (41, 1)},
 'legend': ('@ COVID cases, thousands\n', 'M™ Flu cases, thousands\n'),
 'title': 'COVID vs flu cases, 2020 10\n',
 'x_ticks': ['Jan',
             'Feb',
             'Mar',
             'Apr',
             'May',
             'Jun',
             'Jul.',
             'Aug',
             'Sep',
             'Oct',
             'Nov',
             'Dec.'],
 'x_title': 'Month (2020)\n',
 'y_ticks': [200, 150, 100, 0],
 'y_title': 'Cases of illness (USA)\n'}


In [110]:
for key in feature_extraction:
    if type(feature_extraction[key]) is list or type(feature_extraction[key]) is tuple:
        new_list = [""]*len(feature_extraction[key])
        for i in range(len(feature_extraction[key])):
            element = str(feature_extraction[key][i])
            for character in element:
                if character.isalnum() == False and character != "," and character != "." and character != "(" and character != ")" and character != "!" and character != "?" and character != " ":
                    element = element.replace(character,"")
                new_list[i] = element
        feature_extraction[key] = new_list
    if type(feature_extraction[key]) is str:
        newString = feature_extraction[key]
        for character in feature_extraction[key]:
            if character.isalnum() == False and character != "," and character != "." and character != "(" and character != ")" and character != "!" and character != "?" and character != " ":
                newString = newString.replace(character,"")
        feature_extraction[key] = newString

In [111]:
pprint.pprint(feature_extraction)

{'bottom_y_tick': 0,
 'data': {'Apr': (18, 1),
          'Aug': (61, 1),
          'Dec.': 159,
          'Feb': (1, 25),
          'Jan': (1, 16),
          'Jul.': (30, 1),
          'Jun': (21, 1),
          'Mar': (1, 16),
          'May': (27, 1),
          'Nov': (78, 1),
          'Oct': (41, 1),
          'Sep': (41, 1)},
 'legend': [' COVID cases, thousands', 'M Flu cases, thousands'],
 'title': 'COVID vs flu cases, 2020 10',
 'x_ticks': ['Jan',
             'Feb',
             'Mar',
             'Apr',
             'May',
             'Jun',
             'Jul.',
             'Aug',
             'Sep',
             'Oct',
             'Nov',
             'Dec.'],
 'x_title': 'Month (2020)',
 'y_ticks': ['200', '150', '100', '0'],
 'y_title': 'Cases of illness (USA)'}


In [36]:
#heuristic analysis
heuristic_misleaders = []
x_ticks = features["x_ticks"]
y_ticks = features["y_ticks"]
#truncated y axis
if (features["bottom_y_tick"]) != 0:
    heuristic_misleaders.append("Truncated y axis")
#nonlinear x axis
delta = []
if type(x_ticks[0])=="int":
    for i in range(len(x_ticks)-1):
        delta.append(int(x_ticks[i+1])-int(x_ticks[i]))
    avgDelta = 0
    for i in range(len(delta)):
        avgDelta=avgDelta+delta[i]
    avgDelta = avgDelta//len(delta)
    for i in range(len(delta)-1):
        if delta[i] > avgDelta + 10 or delta[i] < avgDelta - 10:
            heuristic_misleaders.append("Nonlinear x axis")
            break
    if avgDelta <0:
        heuristic_misleaders.append("Backwards x axis")
#nonlinear y axis
delta = []
for i in range(len(y_ticks)-1):
    delta.append(int(y_ticks[i+1])-int(y_ticks[i]))
avgDelta = 0
for i in range(len(delta)):
    avgDelta=avgDelta+delta[i]
avgDelta = avgDelta//len(delta)
for i in range(len(delta)-1):
    if delta[i] > avgDelta + 2 or delta[i] < avgDelta - 2:
        heuristic_misleaders.append("Nonlinear y axis")
        break
if avgDelta > 0:
    heuristic_misleaders.append("Backwards y axis")


In [56]:
print(heuristic_misleaders)

['Nonlinear y axis']


In [ ]:
#import os
#import openai
#openai.api_type = "azure"
#openai.api_base = "https://jha-openai.openai.azure.com/" 
#openai.api_version = "2023-05-15"
#openai.api_key = openai.api_key = "2c6aa030a1c54d6fbb83335870dddcf6"

#response = openai.ChatCompletion.create(
#    engine="test4",
#    messages=[
#        {"role": "user", "content": "Here are the features of a graph I found. The title is \"The US Population is shrinking\". Its x axis is titled \"year\". The y axis measures two variables, \"US population, in millions\", and \"US emmigration, in millions\". Here is a list of the graph's data.\nData point 1: \"year\" = 2000, \"US population, in millions\" = 500, \"US emmigration, in millions\" = 0\nData point 2: \"year\" = 2005, \"US population, in millions\" = 400, \"US emmigration, in millions\" = 100\nData point 3:\"year\" = 2010, \"US population, in millions\" = 300, \"US emmigration, in millions\" = 200\nData point 4: \"year\" = 2015, \"US population, in millions\" = 200, \"US emmigration, in millions\" = 300\nData point 5: \"year\" = 2020, \"US population, in millions\" = 100, \"US emmigration, in millions\" = 400\nGiven this description of the graph, summarize what the data is suggesting."},
#        {"role": "user", "content": "Repeat exactly what I just said, but correct any obvious spelling mistakes."},
#    ]
#)
#print(response['choices'][0]['message']['content'])

In [ ]:
#import os
#import openai
#openai.api_type = "azure"
#openai.api_base = "https://jha-openai.openai.azure.com/" 
#openai.api_version = "2023-05-15"
#openai.api_key = openai.api_key = "2c6aa030a1c54d6fbb83335870dddcf6"

#response = openai.ChatCompletion.create(
#    engine="test4",
#    messages=[
#        {"role": "user", "content": "Here are the features of a graph I found. The title is \"The US Population is shrinking\". Its x axis is titled \"year\". The y axis measures two variables, \"US population, in millions\", and \"US emmigration, in millions\". Here is a list of the graph's data.\nData point 1: \"year\" = 2000, \"US population, in millions\" = 500, \"US emmigration, in millions\" = 0\nData point 2: \"year\" = 2005, \"US population, in millions\" = 400, \"US emmigration, in millions\" = 100\nData point 3:\"year\" = 2010, \"US population, in millions\" = 300, \"US emmigration, in millions\" = 200\nData point 4: \"year\" = 2015, \"US population, in millions\" = 200, \"US emmigration, in millions\" = 300\nData point 5: \"year\" = 2020, \"US population, in millions\" = 100, \"US emmigration, in millions\" = 400\nGiven this description of the graph, summarize what the data is suggesting."},
#        {"role": "user", "content": "What about this graph, if anything, might mislead an uninformed reader?"},
#        {"role": "user", "content": "Based on your previous answer, is this graph misleading? Please answer yes or no"}
#    ]
#)
#print(response['choices'][0]['message']['content'])

In [1]:
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://jha-openai.openai.azure.com/" 
openai.api_version = "2023-05-15"
openai.api_key = openai.api_key = "2c6aa030a1c54d6fbb83335870dddcf6"
response = openai.ChatCompletion.create(
    engine="test4",
    messages=[
        {"role": "user", "content": "what is welfare economics?"},
    ]
)
print(response['choices'][0]['message']['content'])

Welfare economics is a branch of economics that studies how economic policies can improve the welfare of individuals and society as a whole. It focuses on the analysis of how the allocation of resources affects the well-being of people and examines the efficiency and equity of different market outcomes. The ultimate goal of welfare economics is to identify policies that can lead to a more efficient and fair distribution of resources, and ultimately, to improve the overall quality of life for everyone.
